In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,611 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,449 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages 

In [2]:
from pyspark.sql import SparkSession  #INICIAR SPARK
spark = SparkSession.builder \
               .appName('ml') \
               .getOrCreate()

In [10]:
df = spark.read.csv('/Movies.csv', header=True, inferSchema=True) #carga del dataset

In [4]:
df.printSchema() #Vistazo y tipos de dato de las columnas

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- budget: integer (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)



In [5]:
df.show() #vistazo del DF

+--------+--------------------+------------+----------+--------+------------+----------+-----+---------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|vote_average|vote_count|  status|release_date|   revenue|adult|   budget|  imdb_id|original_language|      original_title|            overview|popularity|              genres|production_companies|production_countries|    spoken_languages|
+--------+--------------------+------------+----------+--------+------------+----------+-----+---------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
| 4520010|           Inception|       8.364|     34495|Released|   7/15/2010| 825532764|false|160000000|tt1375666|               en|           Inception|"Cobb a skilled t...|    83.952|Actio

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [11]:
df.count() #Numero de filas del df

1048575

In [12]:
from pyspark.sql.functions import count, when

# Contar los valores nulos en todas las columnas
null_counts = df.agg(*[(count(when(col(c).isNull(), c)).alias(c)) for c in df.columns])
null_counts.show()

+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------+--------------------+--------------------+----------------+
|movie_id|title|vote_average|vote_count|status|release_date|revenue|adult|budget|imdb_id|original_language|original_title|overview|popularity|genres|production_companies|production_countries|spoken_languages|
+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------+--------------------+--------------------+----------------+
|       0|   16|           0|         0|     0|      149422|      0|    0|     0| 471214|                0|            15|  198224|         0|402283|              560686|              446049|          428440|
+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------+---------------

In [13]:
df_cleaned = df.dropna() #Eliminar valores nulos

In [14]:

# Contar valores nulos en todas las columnas del archivo depurado
null_counts = df_cleaned.agg(*[(count(when(col(c).isNull(), c)).alias(c)) for c in df_cleaned.columns])
null_counts.show()

+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------+--------------------+--------------------+----------------+
|movie_id|title|vote_average|vote_count|status|release_date|revenue|adult|budget|imdb_id|original_language|original_title|overview|popularity|genres|production_companies|production_countries|spoken_languages|
+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------+--------------------+--------------------+----------------+
|       0|    0|           0|         0|     0|           0|      0|    0|     0|      0|                0|             0|       0|         0|     0|                   0|                   0|               0|
+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------+---------------

In [15]:
df_cleaned.count() # Numero de registros, debe ser mucho menor al previo

224173

In [16]:
from pyspark.ml.feature import VectorAssembler


In [17]:
df_cleaned.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- budget: integer (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)



In [18]:
from pyspark.sql.functions import col, to_date

In [19]:
df_cleaned.show()

+--------+--------------------+------------+----------+--------+------------+----------+-----+---------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|vote_average|vote_count|  status|release_date|   revenue|adult|   budget|  imdb_id|original_language|      original_title|            overview|popularity|              genres|production_companies|production_countries|    spoken_languages|
+--------+--------------------+------------+----------+--------+------------+----------+-----+---------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
| 4520010|           Inception|       8.364|     34495|Released|   7/15/2010| 825532764|false|160000000|tt1375666|               en|           Inception|"Cobb a skilled t...|    83.952|Actio

In [20]:
from pyspark.sql.functions import regexp_replace

df = df_cleaned.withColumn("release_date", regexp_replace(df_cleaned["release_date"], "/","-")) #cambiar el "/" pór "-"

In [21]:
df.printSchema() #la fecha (release_date) sigue siendo un string

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- budget: integer (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)



In [22]:
from pyspark.sql.functions import  lpad


In [23]:
df = df.withColumn("release_date", regexp_replace(lpad(regexp_replace(df["release_date"],
                                                                      '-', '/'),10, "0"),
                                                  '/', '-')) #Rellena con 0 los meses con un sólo digito

In [ ]:
df.show()

+--------+--------------------+------------+----------+--------+------------+----------+-----+---------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|vote_average|vote_count|  status|release_date|   revenue|adult|   budget|  imdb_id|original_language|      original_title|            overview|popularity|              genres|production_companies|production_countries|    spoken_languages|
+--------+--------------------+------------+----------+--------+------------+----------+-----+---------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
| 4520010|           Inception|       8.364|     34495|Released|  07-15-2010| 825532764|false|160000000|tt1375666|               en|           Inception|"Cobb a skilled t...|    83.952|Actio

In [24]:
df= df.withColumn("release_date", to_date(df["release_date"], "MM-dd-yyyy")) #Convertir a formato fecha

In [25]:
df.printSchema() #Comprobar el tipo de dato

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- revenue: long (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- budget: integer (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)



In [26]:
from pyspark.sql.functions import year


In [27]:
df = df.withColumn('año', year(df['release_date'])) #generar una columna sólo del año de lanzamiento


In [28]:
df.show()

+--------+--------------------+------------+----------+--------+------------+----------+-----+---------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+----+
|movie_id|               title|vote_average|vote_count|  status|release_date|   revenue|adult|   budget|  imdb_id|original_language|      original_title|            overview|popularity|              genres|production_companies|production_countries|    spoken_languages| año|
+--------+--------------------+------------+----------+--------+------------+----------+-----+---------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+----+
| 4520010|           Inception|       8.364|     34495|Released|  2010-07-15| 825532764|false|160000000|tt1375666|               en|           Inception|"Cobb a skilled t...| 

In [29]:
from pyspark.ml.feature import VectorAssembler, ChiSqSelector

Para poder usar de manera adecuada la fecha como input la generé una columna sólo con el año y divido las peliculas en si son para adultos "1" o si no lo son "0"

In [30]:
df = df.withColumn("adult", when(df["adult"] == "False", 0).otherwise(1))


In [31]:
df = df.withColumn("status", when(df["status"] == "Released", 1).otherwise(0))

In [32]:
df.show(5)

+--------+---------------+------------+----------+------+------------+----------+-----+---------+---------+-----------------+---------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+----+
|movie_id|          title|vote_average|vote_count|status|release_date|   revenue|adult|   budget|  imdb_id|original_language| original_title|            overview|popularity|              genres|production_companies|production_countries|    spoken_languages| año|
+--------+---------------+------------+----------+------+------------+----------+-----+---------+---------+-----------------+---------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+----+
| 4520010|      Inception|       8.364|     34495|     1|  2010-07-15| 825532764|    0|160000000|tt1375666|               en|      Inception|"Cobb a skilled t...|    83.952|Action Science Fi...|Legendary Picture

In [33]:

# volver a validar fdatos nulos ya que algunas fechas no tienen el dato año
null_count = df.agg(*[(count(when(col(c).isNull(), c)).alias(c)) for c in df.columns])
null_count.show()

+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------+--------------------+--------------------+----------------+---+
|movie_id|title|vote_average|vote_count|status|release_date|revenue|adult|budget|imdb_id|original_language|original_title|overview|popularity|genres|production_companies|production_countries|spoken_languages|año|
+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------+--------------------+--------------------+----------------+---+
|       0|    0|           0|         0|     0|         497|      0|    0|     0|      0|                0|             0|       0|         0|     0|                   0|                   0|               0|497|
+--------+-----+------------+----------+------+------------+-------+-----+------+-------+-----------------+--------------+--------+----------+------

In [34]:
df = df.dropna()

In [39]:
salida = "/Movies_clean" #guardo el archivo para futuros ejercicios
df.write.option("header", "true").csv(salida)

In [40]:
from pyspark.sql.functions import ltrim, StringType


In [41]:
df = df.select([ltrim(col(c)).alias(c) if isinstance(df.schema[c].dataType, StringType) else col(c) for c in df.columns])


In [48]:
df_excluido = df.drop("title","release_date",
                      "original_language","original_title",
                      "overview","genres", "production_companies","production_countries"
                      "spoken_languages")


In [49]:
df_excluido.show(5)

+--------+------------+----------+------+----------+-----+---------+---------+----------+--------------------+--------------------+----+
|movie_id|vote_average|vote_count|status|   revenue|adult|   budget|  imdb_id|popularity|production_countries|    spoken_languages| año|
+--------+------------+----------+------+----------+-----+---------+---------+----------+--------------------+--------------------+----+
| 4520010|       8.364|     34495|     1| 825532764|    0|160000000|tt1375666|    83.952|United Kingdom Un...|English French Ja...|2010|
| 4520011|       8.417|     32571|     1| 701729206|    0|165000000|tt0816692|   140.241|United Kingdom Un...|             English|2014|
| 4520012|       8.512|     30619|     1|1004558444|    0|185000000|tt0468569|   130.643|United Kingdom Un...|    English Mandarin|2008|
| 4520013|       7.573|     29815|     1|2923706026|    0|237000000|tt0499549|    79.932|United States of ...|     English Spanish|2009|
| 4520014|        7.71|     29166|     1|

In [50]:
df_ex = df_excluido.drop("production_countries","spoken_languages", "imdb_id")

In [51]:
df_ex.show(5) # DF sólo con datos numéricos

+--------+------------+----------+------+----------+-----+---------+----------+----+
|movie_id|vote_average|vote_count|status|   revenue|adult|   budget|popularity| año|
+--------+------------+----------+------+----------+-----+---------+----------+----+
| 4520010|       8.364|     34495|     1| 825532764|    0|160000000|    83.952|2010|
| 4520011|       8.417|     32571|     1| 701729206|    0|165000000|   140.241|2014|
| 4520012|       8.512|     30619|     1|1004558444|    0|185000000|   130.643|2008|
| 4520013|       7.573|     29815|     1|2923706026|    0|237000000|    79.932|2009|
| 4520014|        7.71|     29166|     1|1518815515|    0|220000000|    98.082|2012|
+--------+------------+----------+------+----------+-----+---------+----------+----+
only showing top 5 rows



In [52]:
df_ex.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: integer (nullable = false)
 |-- revenue: long (nullable = true)
 |-- adult: integer (nullable = false)
 |-- budget: integer (nullable = true)
 |-- popularity: double (nullable = true)
 |-- año: integer (nullable = true)



In [53]:
from pyspark.sql.types import IntegerType
df_fin = df_ex.select([col(c).cast(IntegerType()).alias(c) for c in df_ex.columns])


In [54]:
df_fin.show(5) #vistazo a los datos numéricos ya como enteros

+--------+------------+----------+------+-----------+-----+---------+----------+----+
|movie_id|vote_average|vote_count|status|    revenue|adult|   budget|popularity| año|
+--------+------------+----------+------+-----------+-----+---------+----------+----+
| 4520010|           8|     34495|     1|  825532764|    0|160000000|        83|2010|
| 4520011|           8|     32571|     1|  701729206|    0|165000000|       140|2014|
| 4520012|           8|     30619|     1| 1004558444|    0|185000000|       130|2008|
| 4520013|           7|     29815|     1|-1371261270|    0|237000000|        79|2009|
| 4520014|           7|     29166|     1| 1518815515|    0|220000000|        98|2012|
+--------+------------+----------+------+-----------+-----+---------+----------+----+
only showing top 5 rows



In [55]:
df_fin.printSchema() #comprobar que todos los datos sean int

root
 |-- movie_id: integer (nullable = true)
 |-- vote_average: integer (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: integer (nullable = false)
 |-- revenue: integer (nullable = true)
 |-- adult: integer (nullable = false)
 |-- budget: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- año: integer (nullable = true)



In [56]:
inputCols = ["año", "vote_average", "status"]
assembler = VectorAssembler(inputCols=inputCols, outputCol="features")

df_with_features = assembler.transform(df_fin)
df_with_features.select("features").show(10,False)

+----------------+
|features        |
+----------------+
|[2010.0,8.0,1.0]|
|[2014.0,8.0,1.0]|
|[2008.0,8.0,1.0]|
|[2009.0,7.0,1.0]|
|[2012.0,7.0,1.0]|
|[2016.0,7.0,1.0]|
|[2018.0,8.0,1.0]|
|[1999.0,8.0,1.0]|
|[2014.0,7.0,1.0]|
|[1994.0,8.0,1.0]|
+----------------+
only showing top 10 rows



In [57]:
df_with_features.show(4)

+--------+------------+----------+------+-----------+-----+---------+----------+----+----------------+
|movie_id|vote_average|vote_count|status|    revenue|adult|   budget|popularity| año|        features|
+--------+------------+----------+------+-----------+-----+---------+----------+----+----------------+
| 4520010|           8|     34495|     1|  825532764|    0|160000000|        83|2010|[2010.0,8.0,1.0]|
| 4520011|           8|     32571|     1|  701729206|    0|165000000|       140|2014|[2014.0,8.0,1.0]|
| 4520012|           8|     30619|     1| 1004558444|    0|185000000|       130|2008|[2008.0,8.0,1.0]|
| 4520013|           7|     29815|     1|-1371261270|    0|237000000|        79|2009|[2009.0,7.0,1.0]|
+--------+------------+----------+------+-----------+-----+---------+----------+----+----------------+
only showing top 4 rows



In [58]:
df_with_features.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- vote_average: integer (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: integer (nullable = false)
 |-- revenue: integer (nullable = true)
 |-- adult: integer (nullable = false)
 |-- budget: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- año: integer (nullable = true)
 |-- features: vector (nullable = true)



In [59]:
selector = ChiSqSelector(numTopFeatures = 3, featuresCol = "features", labelCol="adult",
                         outputCol="selected_features")
df_sel = selector.fit(df_with_features).transform(df_with_features)
df_sel.show(5)

+--------+------------+----------+------+-----------+-----+---------+----------+----+----------------+-----------------+
|movie_id|vote_average|vote_count|status|    revenue|adult|   budget|popularity| año|        features|selected_features|
+--------+------------+----------+------+-----------+-----+---------+----------+----+----------------+-----------------+
| 4520010|           8|     34495|     1|  825532764|    0|160000000|        83|2010|[2010.0,8.0,1.0]| [2010.0,8.0,1.0]|
| 4520011|           8|     32571|     1|  701729206|    0|165000000|       140|2014|[2014.0,8.0,1.0]| [2014.0,8.0,1.0]|
| 4520012|           8|     30619|     1| 1004558444|    0|185000000|       130|2008|[2008.0,8.0,1.0]| [2008.0,8.0,1.0]|
| 4520013|           7|     29815|     1|-1371261270|    0|237000000|        79|2009|[2009.0,7.0,1.0]| [2009.0,7.0,1.0]|
| 4520014|           7|     29166|     1| 1518815515|    0|220000000|        98|2012|[2012.0,7.0,1.0]| [2012.0,7.0,1.0]|
+--------+------------+---------

In [ ]:
spark.stop()
